In [15]:
import numpy as np
import pandas as pd
import pickle

In [3]:
freq_matrix = pickle.load(open('data/freq_matrix.pickle', 'rb'))
ingredients = pickle.load(open('data/ingredients.pickle', 'rb'))
master = pickle.load(open('data/tasteline.pickle', 'rb'))
blacklist = open('data/blacklist').read().lower().split("\n")

# Create frequency dataframe

In [8]:
freq_matrix.shape

(2500, 2500)

In [7]:
master_df = pd.DataFrame(master)
print(master_df.shape)
master_df.head()

(12712, 11)


,aug_ingredients,categories,cooking_time,id,image,ingredients,ingredients_true,instructions,nutr,tags,title
0,"[matvete, majskyckling, halloumi, olivolja, gr...","[sallad, steka, varmrätt, matvete, majskyckling]",15 minuter,http://www.tasteline.com/recept/sandras-ljumna...,http://cdn1.tasteline.com/Ljummen-kycklingsall...,"[matvete, majskyckling, halloumi, olivolja, gr...","[matvete, majskyckling(ar), halloumi, olivolja...","Dressing \n\n1. Blanda samman matyoghurt, citr...","[Kcal: 681 , Protein: 24 gram, Fett: 37 gram, ...","{'main_type': ['mat'], 'recipe_type': ['sallad...",Sandras ljumna kycklingsallad med halloumi och...
1,"[majskyckling, salt, svartpeppar, smör, vitlök...","[i ugn, varmrätt, majskyckling, morot]",1 timme,http://www.tasteline.com/recept/sandras-grilla...,http://cdn1.tasteline.com/Helstekt-majskycklin...,"[majskyckling, salt, svartpeppar, smör, vitlök...","[majskyckling(ar), salt, svartpeppar, smör, vi...",Kyckling och grönsaker\n\n1. Sätt ugnen på 175...,"[Kcal: 465 , Protein: 35 gram, Fett: 17 gram, ...","{'main_type': ['mat'], 'recipe_type': [], 'kit...",Sandras grillade majskyckling med apelsin och ...
2,"[kantareller, smör, vatten, grädde, svampfond,...","[fest, koka, sås, tillbehör, kantareller]",20 minuter,http://www.tasteline.com/recept/kantarellsas-3/,http://cdn1.tasteline.com/svampsas.jpg,"[kantareller, smör, vatten, grädde, svampfond,...","[kantareller, smör, vatten, vispgrädde, svampf...",1. Lägg svampen i en sil och låt vätskan rinna...,[],"{'main_type': ['mat'], 'recipe_type': ['sås'],...",Kantarellsås
3,"[fläskfärs, salt, peppar, glasnudlar, paprika,...","[gi, glutenfri, sallad, varmrätt, fläskfärs, g...",,http://www.tasteline.com/recept/yum-woon-sen-g...,http://cdn1.tasteline.com/Yum-Woon-Sen-683x102...,"[fläskfärs, salt, peppar, glasnudlar, paprika,...","[fläskfärs, salt, peppar, glasnudlar, paprika(...",1. Blötlägg glasnudlarna i kallt vatten. Skär ...,"[Kcal: 438.3 , Protein: 21.7 gram, Fett: 27 gr...","{'main_type': ['mat'], 'recipe_type': ['sallad...",Yum Woon Sen – Glasnudelsallad med fläskfärs
4,"[vita bönor, sojabönor, vitlök, ingefära, chil...","[biffar, steka, varmrätt, vegetariskt, vita bö...",45 minuter,http://www.tasteline.com/recept/asiatiska-bonb...,http://cdn1.tasteline.com/Asiatiskabonbiffar_m...,"[vita bönor, sojabönor, vitlök, ingefära, chil...","[vita bönor, sojabönor, vitlök(ar), ingefära, ...",1. Sätt ugnen på 175 grader.\n\nAsiatiska bönb...,"[Kcal: 392 , Protein: 14 gram, Fett: 30 gram, ...","{'main_type': ['mat'], 'recipe_type': ['biffar...",Asiatiska bönbiffar med glasnudelsallad


In [257]:
master_df.to_pickle('data/master_df.pkl')

In [82]:
%%time
ingredient_df = pd.get_dummies(master_df.aug_ingredients.apply(pd.Series).stack()).sum(level=0)
freq_df = ingredient_df.astype(int).T.dot(ingredient_df.astype(int))

CPU times: user 1min, sys: 8.46 s, total: 1min 8s
Wall time: 1min 15s


### Make MAIN column

In [85]:
tag_df = master_df.tags.apply(lambda x: pd.Series(x))
tag_df.head()

,main_type,recipe_type,kitchen,occasion,main_ingr,dish_type,author
0,[mat],[sallad],[],[],"[matvete, majskyckling]",[varmrätt],[]
1,[mat],[],[],[],"[majskyckling, morot]",[varmrätt],[]
2,[mat],[sås],[],[fest],[kantareller],[tillbehör],[mia troberg]
3,[mat],[sallad],[],[],"[fläskfärs, glasnudlar]",[varmrätt],[mia troberg]
4,[mat],[biffar],[asien],[],"[vita bönor, glasnudlar]",[varmrätt],[mari bergman]


In [86]:
main_freqs = pd.Series(ingredient for recipe in tag_df.main_ingr for ingredient in recipe).value_counts()
main_freqs = main_freqs.to_frame(name='MAIN')

freq_df = freq_df.merge(main_freqs, how='left', left_index=True, right_index=True).fillna(0)
freq_df.loc[:, 'MAIN'] = freq_df['MAIN'].astype(int)

freq_df.head()

,5-minuters sill,5:2,</span>,HP-sås,abba bredbart tonfiskpålägg,abba middagsklart citron & dillsås för ugnsbakad lax,abba middagsklart citronsås för ugnsbakad lax,abba middagsklart dillsås för ugnsbakad torsk,abba middagsklart fetaostpesto för ugnsbakad lax,abba middagsklart mild thaigryta för lax,...,ölkorv,öringar,örtagårdskrydda,örtbiffar,örter,örtkryddor,örtsalt,örtsås,örttärning,MAIN
5-minuters sill,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,14
5:2,0,51,0,0,0,0,0,0,0,0,...,0,0,0,0,4,1,0,0,0,0
</span>,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HP-sås,0,0,0,19,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
abba bredbart tonfiskpålägg,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Make CATEGORY column

In [107]:
cat_freqs = pd.Series(category for recipe in master_df.categories for category in recipe).value_counts()
cat_freqs = cat_freqs.to_frame(name='CATEGORY')

freq_df = freq_df.merge(cat_freqs, how='left', left_index=True, right_index=True).fillna(0)
freq_df.loc[:, 'CATEGORY'] = freq_df['CATEGORY'].astype(int)

freq_df.head()

,5-minuters sill,5:2,</span>,HP-sås,abba bredbart tonfiskpålägg,abba middagsklart citron & dillsås för ugnsbakad lax,abba middagsklart citronsås för ugnsbakad lax,abba middagsklart dillsås för ugnsbakad torsk,abba middagsklart fetaostpesto för ugnsbakad lax,abba middagsklart mild thaigryta för lax,...,öringar,örtagårdskrydda,örtbiffar,örter,örtkryddor,örtsalt,örtsås,örttärning,MAIN,CATEGORY
5-minuters sill,14,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,14,14
5:2,0,51,0,0,0,0,0,0,0,0,...,0,0,0,4,1,0,0,0,0,51
</span>,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HP-sås,0,0,0,19,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
abba bredbart tonfiskpålägg,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


### Pickle

In [108]:
freq_df.to_pickle('data/freq_df.pkl')

In [4]:
freq_df = pd.read_pickle('data/freq_df.pkl')

# Train ingr2vec

In [5]:
from gensim.models import Word2Vec, FastText
from multiprocessing import cpu_count

In [8]:
%%time
d = 50
min_count = master_df.shape[0] * 0.005
iterations = 300
max_len = pd.Series([ingr for ingr in entry if freq_df.loc[ingr, ingr] > min_count] for entry in master_df.ingredients).apply(len).max()
ingr2vec = Word2Vec(
    pd.Series([ingr for ingr in entry if freq_df.loc[ingr, ingr] > min_count] for entry in master_df.ingredients),
    size=d,
    window=max_len,
    min_count=min_count,
    workers=cpu_count(),
    iter=iterations,
    hs=0,
    negative=10,
    sg=0,
    cbow_mean=1,
    # min_n=1,
    # max_n=5,
    # word_ngrams=0
)

CPU times: user 48.6 s, sys: 481 ms, total: 49.1 s
Wall time: 21.3 s


# Compare to old

In [10]:
i2v = Word2Vec.load('data/ingr2vec_model_200')

In [11]:
from pprint import pprint
tmp_ingr = 'spetskål'
try:
    pprint(ingr2vec.wv.most_similar(tmp_ingr))
except KeyError:
    pass
i2v.wv.most_similar(tmp_ingr)

[('konserverade tomater', 0.3531292676925659),
 ('sidfläsk', 0.3094513714313507),
 ('ljust muscovadoråsocker', 0.2861067056655884),
 ('kinakål', 0.27282238006591797),
 ('ugnsfolie', 0.26009002327919006),
 ('parmaskinka', 0.25848180055618286),
 ('senap', 0.23832416534423828),
 ('vinsyra', 0.22454260289669037),
 ('korianderfrön', 0.22147688269615173),
 ('körvel', 0.22041890025138855)]

In [12]:
from pprint import pprint
tmp_ingr = 'grädde'
try:
    pprint(ingr2vec.wv.most_similar(tmp_ingr))
except KeyError:
    pass
i2v.wv.most_similar(tmp_ingr)

[('crème fraiche', 0.6429681181907654),
 ('mjölk', 0.5330901741981506),
 ('smör', 0.5154533982276917),
 ('smördeg', 0.34582921862602234),
 ('konjak', 0.3412764072418213),
 ('vitt vin', 0.31079617142677307),
 ('worcestershiresås', 0.29263901710510254),
 ('vin', 0.27357354760169983),
 ('vitpeppar', 0.25894588232040405),
 ('risottoris', 0.25805652141571045)]


[('crème fraiche', 0.5170435905456543),
 ('mjölk', 0.40699249505996704),
 ('smördeg', 0.26500970125198364),
 ('rotselleri', 0.2405264973640442),
 ('gräddfil', 0.2375752329826355),
 ('grön paprika', 0.20780640840530396),
 ('matyoghurt', 0.2038072645664215),
 ('päron', 0.19016596674919128),
 ('kikärtor', 0.1850806623697281),
 ('olja', 0.18368639051914215)]

### Pickle

In [23]:
ingr2vec.save('models/ingr2vec.pkl')

# Re-create w2v NN

In [13]:
ingr2vec.predict_output_word(['tomater', 'basilika'])

[('mozzarella', 0.07980521),
 ('salami', 0.038896907),
 ('fetaost', 0.036596425),
 ('oliver', 0.031980548),
 ('halloumi', 0.023775203),
 ('pasta', 0.0223485),
 ('spaghetti', 0.020400004),
 ('pinjenötter', 0.019764105),
 ('parmesanost', 0.019662797),
 ('balsamvinäger', 0.018149624)]

In [16]:
for i in (np.dot(np.mean([ingr2vec.wv['tomater'], ingr2vec['basilika']], axis=0), ingr2vec.syn1neg.T)).argsort()[-5:][::-1]:
    print(ingr2vec.wv.index2word[i])

mozzarella
salami
fetaost
oliver
halloumi


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  """Entry point for launching an IPython kernel.


In [17]:
import keras
from keras import Sequential, Model
from keras.layers import Dense, Input, Embedding, Lambda

Using TensorFlow backend.


In [31]:
MAX_SEQUENCE_LENGTH = 20

input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

embedding_layer = Embedding(
    ingr2vec.wv.vectors.shape[0] + 1,
    ingr2vec.wv.vectors.shape[1],
    mask_zero=True,
    weights=[np.insert(ingr2vec.wv.vectors, 0, np.zeros(ingr2vec.wv.vectors.shape[1]), axis=0)],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=True
)(input_layer)

embedding_sum = Lambda(lambda x: keras.backend.sum(x, axis=1))(embedding_layer)

prediction_layer = Dense(
    units=ingr2vec.trainables.syn1neg.shape[0],
    weights=(ingr2vec.trainables.syn1neg.T, np.zeros(ingr2vec.trainables.syn1neg.shape[0])),
    activation='softmax'
)(embedding_sum)

model = Model(inputs=input_layer, outputs=prediction_layer)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [55]:
input_vector = [ingr2vec.wv.index2word.index(i) + 1 for i in ['tomater', 'basilika']]
input_vector += [0] * (MAX_SEQUENCE_LENGTH - len(input_vector))
input_vector = np.array([input_vector])
print(input_vector)

pred_vector = model.predict(input_vector)

for i in pred_vector[0].argsort()[::-1][:5]:
    print(ingr2vec.wv.index2word[i])

[[ 9 24  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
mozzarella
salami
fetaost
oliver
halloumi


In [54]:
pred_vector[0].argsort()[::-1][:5]

array([125, 269,  56, 102, 246])

In [50]:
ingr2vec.wv.index2word[125]

'mozzarella'

In [79]:
np.sum([ingr2vec.wv['tomater'], ingr2vec['basilika']], axis=0)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.367746  , -1.9576017 ,  2.586062  , -3.1975234 , -3.034289  ,
        0.11735713,  1.1680373 , -0.74927294,  2.8381839 , -0.72217274,
        1.9394944 , -0.05457371, -0.2582694 ,  2.4131453 , -0.26528752,
       -0.260118  ,  3.4280605 , -1.3490561 , -1.1070995 ,  0.26483995,
       -0.49083918,  0.29337102,  5.1756525 , -7.0854807 , -3.505442  ,
        1.0558871 ,  2.3992946 , -4.642726  ,  0.7179408 ,  1.6968269 ,
       -0.9049691 , -2.0818415 ,  3.168391  , -3.1357505 , -1.9646115 ,
       -2.2484128 , -0.01122311,  0.23000306, -0.8067852 , -4.7634335 ,
       -1.2798321 ,  0.6556438 ,  1.8205528 , -1.1055394 , -2.9045713 ,
       -3.0190172 ,  1.4261283 , -0.17058194, -0.63509786,  0.5172897 ],
      dtype=float32)

In [21]:
embedding_out = keras.backend.function([input_layer], [embedding_sum])
embedding_out([input_vector])[0][0]

array([ 0.367746  , -1.9576017 ,  2.586062  , -3.1975234 , -3.034289  ,
        0.11735713,  1.1680373 , -0.74927294,  2.8381839 , -0.72217274,
        1.9394944 , -0.05457371, -0.2582694 ,  2.4131453 , -0.26528752,
       -0.260118  ,  3.4280605 , -1.3490561 , -1.1070995 ,  0.26483995,
       -0.49083918,  0.29337102,  5.1756525 , -7.0854807 , -3.505442  ,
        1.0558871 ,  2.3992946 , -4.642726  ,  0.7179408 ,  1.6968269 ,
       -0.9049691 , -2.0818415 ,  3.168391  , -3.1357505 , -1.9646115 ,
       -2.2484128 , -0.01122311,  0.23000306, -0.8067852 , -4.7634335 ,
       -1.2798321 ,  0.6556438 ,  1.8205528 , -1.1055394 , -2.9045713 ,
       -3.0190172 ,  1.4261283 , -0.17058194, -0.63509786,  0.5172897 ],
      dtype=float32)

### Pickle

In [24]:
model.save('models/w2v_copy.pkl')

# TEST PY

In [206]:
%load_ext autoreload
%autoreload 2

import nn_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [396]:
model = nn_utils.recreate_w2v_model(ingr2vec)
model.save('models/w2v_copy.pkl')

In [397]:
nn_utils.predict_next([], model, ingr2vec, n=10)

[('ättika', 0.0035587188),
 ('vitkål', 0.0035587188),
 ('yoghurt', 0.0035587188),
 ('majs', 0.0035587188),
 ('laxfilé', 0.0035587188),
 ('mjöl', 0.0035587188),
 ('spenat', 0.0035587188),
 ('chilisås', 0.0035587188),
 ('bakpulver', 0.0035587188),
 ('mynta', 0.0035587188)]

In [398]:
nn_utils.get_embedding(['tomater', 'basilika'], model, ingr2vec)

array([ 0.367746  , -1.9576017 ,  2.586062  , -3.1975234 , -3.034289  ,
        0.11735713,  1.1680373 , -0.74927294,  2.8381839 , -0.72217274,
        1.9394944 , -0.05457371, -0.2582694 ,  2.4131453 , -0.26528752,
       -0.260118  ,  3.4280605 , -1.3490561 , -1.1070995 ,  0.26483995,
       -0.49083918,  0.29337102,  5.1756525 , -7.0854807 , -3.505442  ,
        1.0558871 ,  2.3992946 , -4.642726  ,  0.7179408 ,  1.6968269 ,
       -0.9049691 , -2.0818415 ,  3.168391  , -3.1357505 , -1.9646115 ,
       -2.2484128 , -0.01122311,  0.23000306, -0.8067852 , -4.7634335 ,
       -1.2798321 ,  0.6556438 ,  1.8205528 , -1.1055394 , -2.9045713 ,
       -3.0190172 ,  1.4261283 , -0.17058194, -0.63509786,  0.5172897 ],
      dtype=float32)

In [395]:
nn_utils.train(model, ingr2vec, [], 'lax')

In [399]:
model.save('models/w2v_spec.pkl')

In [248]:
aa = keras.models.load_model('models/w2v_spec.pkl', custom_objects={'k': keras.backend})

In [411]:
np.linalg.norm((1 - 2, 3 - 1))

2.23606797749979

# Master

In [ ]:
def _generate_recipes(chosen):
    # chosen = {"räkor":["räkor"],"löjrom":["löjrom"],"svarta bönor":["svarta bönor"]}
    top_recipes = 5 * [
        {"id": -1, "title": "", "image": "", "instructions": "", "ingredients": [],
        "cooking_time": "", "categories": [], "nutr": "", "tags": [], "score": -1,
        "substitutions": []}
    ]
    # Prioritize actual choices over alternatives
    original_choices = list(chosen.keys())
    print("Actual choices: {}\n".format(original_choices))
    # Loop over all recipes
    for recipe in master:
        best_over_combos = -1
        substitutions = []
        # Loop over all combinations of alternative ingredients
        for combo in list(itertools.product(*chosen.values())):
            temp_score = 0
            temp_substitutions = []
            for c in combo:
                if c in recipe["aug_ingredients"]:
                    try:
                        # Base score on alternatives' similarity to original choices
                        temp_index = np.argmax([i2v.ingr2vec.similarity(c, s) for s in original_choices])
                        temp_score += i2v.ingr2vec.similarity(c, original_choices[temp_index])
                        if original_choices[temp_index] != c:
                            temp_substitutions.append([c, original_choices[temp_index]])
                    except KeyError:
                        temp_score += 1
            if temp_score > best_over_combos:
                best_over_combos = temp_score
                substitutions = temp_substitutions
        # TODO: weight in if there's an author
        # Add to top recipes if better than lowest score and sort list
        if best_over_combos > top_recipes[0]["score"]:
            recipe["score"] = best_over_combos
            recipe["substitutions"] = substitutions
            top_recipes[0] = recipe
            top_recipes = sorted(top_recipes, key=itemgetter("score"))
    top_recipes.reverse()
    for t in top_recipes:
        t["score"] = round(t["score"], 2)
    return top_recipes


In [259]:
chosen = ['pasta', 'bacon', 'spenat']

In [287]:
master_df['matches'] = master_df.ingredients.apply(lambda recipe: len(list(set(chosen) & set(recipe))))

In [290]:
master_df.sort_values('matches', ascending=False)[:10]

,aug_ingredients,categories,cooking_time,id,image,ingredients,ingredients_true,instructions,nutr,tags,title,matches
4423,"[pasta, spenat, olivolja, bacon, parmesanost, ...","[koka, pasta, sallad, varmrätt, pasta, bacon]",,http://www.tasteline.com/recept/pastasallad-me...,http://cdn1.tasteline.com/Food_missing-1280x83...,"[pasta, spenat, olivolja, bacon, parmesanost, ...","[pasta, spenat, olivolja, bacon, parmesanost, ...",1. Koka pastan al dente enligt anvisning. Spol...,[],"{'main_type': ['mat'], 'recipe_type': ['pasta'...",Pastasallad med paprika och bacon,3
4635,"[vita bönor, pasta, smör, rosmarin, salvia, pe...","[koka, soppa, varmrätt, vegetariskt]",30 minuter,http://www.tasteline.com/recept/minestra-di-ve...,http://cdn1.tasteline.com/minestra_di_verdure.jpg,"[vita bönor, pasta, smör, rosmarin, salvia, pe...","[vita bönor, pasta, smör, rosmarin, salvia, pe...",2. Tillagning 35 minuter. Blötläggning av böno...,[],"{'main_type': ['mat'], 'recipe_type': ['soppa'...",Minestra di verdure,2
49,"[vitlök, schalottenlök, olivolja, soltorkade t...","[koka, pasta, varmrätt, soltorkade tomater, pa...",,http://www.tasteline.com/recept/kramig-pasta-m...,http://cdn1.tasteline.com/Kramig-pasta-med-sol...,"[vitlök, schalottenlök, olivolja, soltorkade t...","[vitlöksklyfta(or), schalottenlök(ar), olivolj...",Tillbehör\n\n1. Koka pastan i saltat vatten en...,"[Kcal: 606 , Protein: 19 gram, Fett: 30 gram, ...","{'main_type': ['mat'], 'recipe_type': ['pasta'...","Krämig pasta med soltorkade tomater, gröna ärt...",2
12163,"[älgfärs, crème fraiche, buljong, ost, bacon, ...","[i ugn, låda, varmrätt, älgfärs, pasta, pasta]",,http://www.tasteline.com/recept/gastlada-pa-pa...,http://cdn1.tasteline.com/Food_missing-1280x83...,"[älgfärs, crème fraiche, buljong, ost, bacon, ...","[älgfärs, crème fraiche, buljong, ost, bacon, ...",1. Sätt ugnen på 225 grader. Bryn färs och bac...,[],"{'main_type': ['mat'], 'recipe_type': ['låda',...",Gästlåda på pasta och älgfärs,2
3307,"[pasta, kalkonkassler, gröna ärtor, vitlök, ci...","[pasta, sallad, steka, varmrätt, pasta, kalkon...",15 minuter,http://www.tasteline.com/recept/artpestoslunga...,http://cdn1.tasteline.com/artpestoslungad_past...,"[pasta, kalkonkassler, gröna ärtor, vitlök, ci...","[pasta, kalkonkassler, gröna ärtor, vitlökskly...",1. Koka pasta enligt anvisning på paketet. Kok...,[],"{'main_type': ['mat'], 'recipe_type': ['pasta'...",Ärtpestoslungad pastasallad med kalkonkassler,2
11815,"[tonfisk, rödlök, bacon, crème fraiche, pasta,...","[pasta, steka, varmrätt, tonfisk, pasta]",,http://www.tasteline.com/recept/pasta-med-het-...,http://cdn1.tasteline.com/Food_missing-1280x83...,"[tonfisk, rödlök, bacon, crème fraiche, pasta]","[tonfisk, rödlök(ar), bacon, crème fraiche, pa...",1. Sätt ugnen på 220 grader. Klipp bacon i små...,[],"{'main_type': ['mat'], 'recipe_type': ['pasta'...",Pasta med het tonfiskröra,2
102,"[pasta, rödlök, vitlök, zucchini, bacon, olivo...","[gratäng, i ugn, varmrätt]",45 minuter,http://www.tasteline.com/recept/pastagratang-m...,http://cdn1.tasteline.com/Pastagratang_med_bac...,"[pasta, rödlök, vitlök, zucchini, bacon, olivo...","[pasta, rödlök, vitlök, zucchini, bacon, olivo...",1. Sätt ugnen på 200 grader.\n\n7. Servera pas...,"[Kcal: 742 , Protein: 33 gram, Fett: 37 gram, ...","{'main_type': ['mat'], 'recipe_type': ['gratän...","Pastagratäng med bacon, ricotta och zucchini",2
5911,"[rödbeta, kycklingfilé, chilisås, rapsolja, vi...","[steka, varmrätt, kycklingfilé, bacon]",,http://www.tasteline.com/recept/chilikyckling-...,http://cdn1.tasteline.com/Food_missing-1280x83...,"[rödbeta, kycklingfilé, chilisås, rapsolja, vi...","[rödbetor, kycklingfilé(er), chilisås, soja, v...","1. Koka betorna med skal tills de är mjuka, sp...",[],"{'main_type': ['mat'], 'recipe_type': [], 'kit...","Chilikyckling med färska betor, bacon och getost",2
4174,"[pasta, bacon, ädelost, tomater, paprika, gurk...","[sallad, steka, student, bacon, ädelost, varmr...",30 minuter,http://www.tasteline.com/re

In [284]:
[c.to_dict() for _, c in cc.iterrows()]

[{'aug_ingredients': ['pasta',
   'spenat',
   'olivolja',
   'bacon',
   'parmesanost',
   'oliver',
   'paprika',
   'vitlök',
   'basilika',
   'salt',
   'peppar',
   'koka',
   'sallad',
   'varmrätt'],
  'categories': ['koka', 'pasta', 'sallad', 'varmrätt', 'pasta', 'bacon'],
  'cooking_time': '',
  'id': 'http://www.tasteline.com/recept/pastasallad-med-paprika-och-bacon/',
  'image': 'http://cdn1.tasteline.com/Food_missing-1280x838.jpg',
  'ingredients': ['pasta',
   'spenat',
   'olivolja',
   'bacon',
   'parmesanost',
   'oliver',
   'paprika',
   'vitlök',
   'basilika',
   'salt',
   'peppar'],
  'ingredients_true': ['pasta',
   'spenat',
   'olivolja',
   'bacon',
   'parmesanost',
   'oliver',
   'paprika',
   'vitlöksklyfta(or)',
   'basilikablad',
   'salt',
   'peppar'],
  'instructions': '1. Koka pastan al dente enligt anvisning. Spola av i kallt vatten.\n\n2. Fräs spenat med vitlöken i olivolja någon minut. Lägg åt sidan. \n\n3. Bryn/knaperstek baconbitarna och låt r

In [282]:
[c for _, c in cc.iterrows()]

[aug_ingredients     [pasta, spenat, olivolja, bacon, parmesanost, ...
 categories              [koka, pasta, sallad, varmrätt, pasta, bacon]
 cooking_time                                                         
 id                  http://www.tasteline.com/recept/pastasallad-me...
 image               http://cdn1.tasteline.com/Food_missing-1280x83...
 ingredients         [pasta, spenat, olivolja, bacon, parmesanost, ...
 ingredients_true    [pasta, spenat, olivolja, bacon, parmesanost, ...
 instructions        1. Koka pastan al dente enligt anvisning. Spol...
 nutr                                                               []
 tags                {'main_type': ['mat'], 'recipe_type': ['pasta'...
 title                               Pastasallad med paprika och bacon
 Name: 4423, dtype: object,
 aug_ingredients     [spenat, bacon, äggula, smör, salt, svartpeppa...
 categories                             [i ugn, paj, varmrätt, spenat]
 cooking_time                                    

In [310]:
min_sim = 0.6
chosen_sim = [[(c, 1.0)] + ingr2vec.wv.most_similar(c, topn=10) for c in chosen]
chosen_sim = [[(sim_ingr, score) for sim_ingr, score in ingr_group if score >= min_sim] for ingr_group in chosen_sim]
chosen_sim

[[('pasta', 1.0),
  ('spaghetti', 0.7833353877067566),
  ('tagliatelle', 0.6923873424530029),
  ('lasagneplattor', 0.6637042760848999),
  ('ris', 0.6409581303596497)],
 [('bacon', 1.0)],
 [('spenat', 1.0), ('bladspenat', 0.742744505405426)]]

In [381]:
import random

In [385]:
random.sample(list(ingr2vec.wv.vocab), 5)

['kokosmjölk', 'curry', 'olja', 'blåmusslor', 'rosépeppar']

# Test app

In [355]:
import app

In [368]:
# app._generate_recipes(['pasta', 'spenat', 'bacon'])

In [425]:
for i in range(10):
    bb = app._generate_suggestions([], None, 10, 560, 966)
    # app.train([], 'pasta')

ValueError: Sample larger than population or is negative

In [437]:
bb = app._generate_suggestions([], None, 10, 560, 966)

In [440]:
np.linalg.norm((4, 3))

5.0